In [8]:
import numpy as np # Library with mathematical computation tools
# import itertools # Iterative library with combinatoric tools if computing the whole cartesian product
import math # To use floor


''' Define auxiliary functions to:
    - get integral number.
    - Generate a filtered array of elements of cartesian product on creation.
    - Extend a list to length n.
    - Filter function to filter elements of the cartesian product. 
    - Get the minimum monomials in a poset with < coordinate-wise. 
    - Print a vector as a monomial. '''

# Function to obtain an integral number 
# Used for the number of relevant exceptional divisors and 
# the number of curves in the maximal contact completion 
# string is a string for the elements to count
def getIntegralNumber(string):
    print(string)
    int_number = input()
    while True:
        try:
            int_number = int(int_number)
            break
        except:
            print("You have to type an integer.")
            int_number = input()
    return int_number



# Define a function to substitute itertools.product
# stackoverflow.com/questions/14614004/how-do-i-convert-pythons-itertools-product-library-from-a-list-comprehension-to
def filtered_product(args,filter_fn,number_functions_monomial,Matrix_orders_vanishing):
    pools = map(tuple, args)
    result = [[]]
    for pool in pools:
        new_result = []
        for x in result:
            for y in pool:
                new_val = x+[y]
                if filter_fn(new_val,number_functions_monomial,Matrix_orders_vanishing):
                    new_result.append(x+[y])
        result = new_result
        # print('intermediate result:', result)

    return result



# Extend (or cut) an array to length number_functions_monomial and append 1
# stackoverflow.com/questions/24066904/most-pythonic-way-to-extend-a-list-to-exactly-a-certain-length
def extend(l, number_functions_monomial):
    l.extend([0] * number_functions_monomial)
    l = np.append(l[:number_functions_monomial],1)
    return l

# A filter for the filtered product, jumping number smaller than one
def filter_fn(x,number_functions_monomial,Matrix_orders_vanishing):

    a = [0 for div in range(number_divisors)]
    b = extend(x,number_functions_monomial)

    for div in range(number_divisors): 
        order_van_array = \
        np.append(Matrix_orders_vanishing[div][:number_functions_monomial], \
                  Matrix_orders_vanishing[div][number_functions + 1])

        a[div] = (b @ order_van_array) / Matrix_orders_vanishing[div][number_functions]

    c = np.amin(a)

    if c<2:
        return True

    
# Compute the minimal elements of a set of vectors wrt < coordinate-wise
def min_poset(A):
    B=[]
    for a in A:
        # if B is empty, add a
        if len(B)==0:
            B.append(a)

        else:
            i = 0
            for b in B:
                # if a is < than an element, change it
                if np.all(np.array(a)<=np.array(b)):
                    b=a
                    break
                # if a is bigger, discard it    
                elif np.all(np.array(b)<=np.array(a)):
                    break
                # else set a counter to check if a is non-comparable with B    
                i += 1
                    
            if i == len(B):
                B.append(a)
    
    return B


# Print an array as a monomial, where n is the number of coordinates contributing to such monomial
def print_monomial(n,v):
    
    if np.count_nonzero(v) == 0:
        print('1')
    
    else:
        for exp in range(n):
            if v[exp] != 0:
                print("z_" + str(exp) + "^" + str(int(v[exp])) + " ", end = '')






    
    
''' Obtain values of the generating functions (completion of the curve) over exceptional divisors. '''

def get_curve_data():
    # Get the number of RED
    string_number_divisors = "How many relevant exceptional divisors does the curve have?"
    number_exceptional_divisors = getIntegralNumber(string_number_divisors)


    # Get the number of MCC
    string_number_functions = "How many functions do you need to generate the maximal contact completion of the curve?"
    number_functions = getIntegralNumber(string_number_functions)

    
    # Get the number of functions needed for monomials
    string_number_functions_acc = "Consider the dual graph of the minimal embedded resolution of the curve. " + \
    "How many functions do you need to complete it " + \
    "(one for each exceptional divisor of valency one)?"
    number_functions_acc = getIntegralNumber(string_number_functions_acc)

    # Get the number of components
    string_number_components = \
    "How many components does the curve have?"
    number_components = getIntegralNumber(string_number_components)
    
    # Is the curve reduced?
    string_curve_reduced = \
    print("Is the curve C reduced? y/n")
    bool_curve_red = input()
    
    if bool_curve_red == 'n' or bool_curve_red == 'N':
        components_weight = [int(item) for item in \
                             input("\nEnter the weights of the components C_i of C separated by single spaces\n").split()]
    
        number_functions_monomial = number_functions
        number_divisors = number_exceptional_divisors + number_components
        
    else:
        components_weight = [1 for i in range(number_components)]
        number_functions_monomial = number_functions_acc
        number_divisors = number_exceptional_divisors


    # Get the order of vanishing of curves in the completion along RED and log-discrepancies
    print("\nNow, for each divisor, introduce the orders of vanishing and its log-discrepancy." \
         "\nLet z_0 represent the transversal branch to the curve " + \
          "and the components of the curve be the last " + str(number_components) + \
         ": z_" + str(number_functions - number_components) + ", z_" + str(number_functions - 1) + \
         ". Also, let z_0, ..., z_" + str(number_functions_monomial) + \
          " be the curves needed to complete the dual graph. " + \
          "For each divisor: " + \
          "Introduce the orders of vanishing of the z_i separated by single spaces and press enter, " + \
          "then introduce the log-discrepancy and press enter again." + \
          " \nThe curve is represented by z_" + str(number_functions) + \
          ".\nThe divisors vary from E_1 to E_" + str(number_divisors))

    # the number of columns is number of functions + 2, 
    # since we add the order of vanishing of the whole curve and the log-discrepancy
    if bool_curve_red == 'n' or bool_curve_red == 'N':
        Matrix_orders_vanishing = [[0 for x in range(number_functions + 2)] \
                                   for y in range(number_divisors + number_components)] 
    else:
        Matrix_orders_vanishing = [[0 for x in range(number_functions + 2)] \
                                   for y in range(number_divisors)] 
        
        
    for div in range(number_exceptional_divisors):
        Matrix_orders_vanishing[div][0:number_functions] = \
        [int(item) for item in input("\nEnter the orders of vanishing along E_" + str(div + 1) + "\n").split()]

        Matrix_orders_vanishing[div][number_functions] = \
        np.array(Matrix_orders_vanishing[div][number_functions - number_components : number_functions]) @ \
        np.array(components_weight)

        string_log_discrepancy = "Introduce the log-discrepancy of the divisor E_" + str(div+1)
        Matrix_orders_vanishing[div][number_functions + 1] = getIntegralNumber(string_log_discrepancy)
    
    if bool_curve_red == 'n' or bool_curve_red == 'N':   
        for div in range(number_components):
            Matrix_orders_vanishing[number_exceptional_divisors + div][number_functions - number_components + div] = 1
            Matrix_orders_vanishing[number_exceptional_divisors + div][number_functions] = components_weight[div]
            Matrix_orders_vanishing[number_exceptional_divisors + div][number_functions +  1] = 1

    # Print the collected data as a matrix
    Matrix_orders_vanishing = np.array(Matrix_orders_vanishing)
    print("\nThe matrix of orders of vanishing and log-discrepancies is:")
    print(Matrix_orders_vanishing)


    return (bool_curve_red,number_exceptional_divisors,number_divisors,number_functions,number_functions_monomial,number_components,components_weight,Matrix_orders_vanishing)
    


''' Compute the Milnor numbers of the components and total curve 
    and the number of jumping numbers counted with multiplicity. 
    Milnor(C) = sum(1,r) Milnor(C_i) + sum(i < j) (Ci,Cj) - number_components + 1 .
    number of jn counted with multiplicity = (Milnor(C) - number_components + 1)/2 .'''

def get_Milnor(number_divisors,number_functions,number_functions_monomial,number_components,Matrix_orders_vanishing):
    # Instructions for user
    print("\nNow introduce the number i of the divisor E_i such that it is the last relevant exceptional divisor" +\
        " of the irreducible curve C_i, generated by the germ z_" + str(number_functions - number_components - 1) + "+ i.")

    corresp_comp_red = [0 for x in range(number_components)]
    Milnor_number = [0 for x in range(number_components)]
    number_jn_multiplicity = 0


    for comp in range(number_components):
        # Obtain the correspondence between curves and relevant exceptional divisors.
        string_corresp_comp_red = "Introduce the number chosen for the last relevant exceptional divisor" +\
        " of the irreducible curve C_" + str(comp + 1) + " = V(z_" +\
        str(number_functions - number_components + comp) + "):"

        corresp_comp_red[comp] = getIntegralNumber(string_corresp_comp_red) - 1

        # Compute the corresponding Milnor number as 
        # order of vanishing of the curve - log-discrepancy + 1
        Milnor_number[comp] = \
        Matrix_orders_vanishing[corresp_comp_red[comp]][number_functions - number_components + comp] - \
        Matrix_orders_vanishing[corresp_comp_red[comp]][number_functions + 1] + 1

        number_jn_multiplicity += Milnor_number[comp]
        print("The Milnor number of the curve C_" + str(comp + 1) + " = V(z_" +\
        str(number_functions - number_components + comp) + ") is: " + str(Milnor_number[comp]))

        for comp2 in range(comp+1,number_components):
            number_jn_multiplicity += \
            2 * Matrix_orders_vanishing[corresp_comp_red[comp]][number_functions - number_components + comp2]        


    Milnor_number_C = number_jn_multiplicity - number_components + 1
    print("The Milnor number of the curve C is: " + str(Milnor_number_C))

    number_jn_multiplicity =  int((Milnor_number_C - number_components + 1) / 2)
    print("The amount of jumping numbers counted with multiplicity is: " + str(number_jn_multiplicity))


    return(number_jn_multiplicity)



''' Generate a matrix with exponent values, 
    the vanishing of the corresponding monomial over divisors and 
    minimal vanishing. '''

def get_jumping_numbers(number_exceptional_divisors,number_divisors,number_functions,number_functions_monomial,number_components,components_weight,Matrix_orders_vanishing):
    
    # Get the maximum exponents for each MCC
    bound_exponents = [0 for x in range(number_functions_monomial)]
    
    for func in range(number_functions_monomial):
        bounding_cond = \
        [(Matrix_orders_vanishing[div][number_functions] - Matrix_orders_vanishing[div][number_functions + 1])/\
         Matrix_orders_vanishing[div][func] for div in range(number_exceptional_divisors)]
        #bounding_cond.extend(components_weight)
        bound_exponents[func] = math.ceil(np.amax(bounding_cond)) #+ 1
        
    #if bool_curve_red == 'n' or bool_curve_red == 'n':
    #    bound_exponents.append(components_weight)

    print("\nThe maximum (inclusive) exponents considered for monomials in the z_i are:")
    print(bound_exponents)
    ####################################### components_weight


    # Generate a (np.prod(bound_exponents)) x (number_functions_monomial + number_divisors + 1) matrix for which:
    # first (number_functions) columns represent a monomial in the semi-roots
    # next number_divisors, from number_functions to number_divisors-1, give the jn condition associated to the monomial
    # the last column is the jumping number associated to the considered monomial

    # Generate the list of monomials
    MonomioGen2 = filtered_product(list(map(range,np.array(bound_exponents))),filter_fn,number_functions_monomial,Matrix_orders_vanishing)
    
    MonomioGen = [[0 for x in range(number_functions_monomial + number_divisors + 1)]\
                 for y in range(len(MonomioGen2))]
    
    # compute the jn conditions
    for mon in range(len(MonomioGen)):
        
        MonomioGen[mon][:number_functions_monomial] = MonomioGen2[mon]
        
        for div in range(number_divisors):
            # the next represent the corresponding condition in the divisor E_(div + 1)
            # monomial times (@) order of vanishing of each function over the vanishing of the curve
            monomial_array_ld = np.append(np.array(MonomioGen[mon][:number_functions_monomial]),1)

            order_van_array = np.append(Matrix_orders_vanishing[div][:number_functions_monomial] , \
            Matrix_orders_vanishing[div][number_functions + 1])

            MonomioGen[mon][number_functions_monomial + div] = \
            (monomial_array_ld @ order_van_array) / Matrix_orders_vanishing[div][number_functions]

        # The jumping number is the minimal condition
        MonomioGen[mon][number_functions_monomial + number_divisors] = \
        np.amin(MonomioGen[mon][number_functions_monomial : number_functions_monomial + number_divisors])


    # Turn the matrix into an array for use of numpy functions and sort the matrix by the corresponding jumping number.
    MonomioGen = np.array(MonomioGen)
    MonomioGen = MonomioGen[MonomioGen[:,number_functions_monomial + number_divisors].argsort()]


    # Make a matrix (array) with elements of minimum vanishing < 1.

    # Number of monomials with minimum vanishing along divisors < 1
    number_dif_jn = sum(i < 1 for i in MonomioGen[:,number_functions_monomial + number_divisors])

    # Keep those elements in JumpingNumberList
    JumpingNumberList = MonomioGen[:number_dif_jn,:]

    # Number of different jumping numbers
    number_jn = len(set(JumpingNumberList[:,number_functions_monomial + number_divisors]))


    return (number_dif_jn,number_jn,JumpingNumberList,MonomioGen)
    
    

''' Print the set of jumping numbers < 1, 
    together with the associated monomial and 
    the divisors at which they are attained. '''

def print_jumping_numbers(number_divisors,number_functions,number_functions_monomial,number_components,\
                          Matrix_orders_vanishing,number_jn_multiplicity,number_dif_jn,number_jn,JumpingNumberList,MonomioGen):

    # The log-canonical threshold, 1st jumping number
    print("\n\nThe log-canonical threshold of the curve is ", end = '') 
    print(JumpingNumberList[0][number_functions_monomial + number_divisors], end = '')

    # Print the fractions corresponding to the lct
    divisors = np.where(JumpingNumberList[0][number_functions_monomial : number_functions_monomial + number_divisors] ==\
                        JumpingNumberList[0][number_functions_monomial + number_divisors])
    for supp in divisors[0]:
        jump_fraction = int(round(JumpingNumberList[0][number_functions_monomial + number_divisors] * \
                                  Matrix_orders_vanishing[supp][number_functions]))
        print(" = " + str(jump_fraction) + "/" + str(Matrix_orders_vanishing[supp][number_functions]), end = '')

    # Print divisors in the support of the lct
    if len(divisors) != 1:
        print(" attained at divisors E_", divisors[0]+1, end = '')
    else:
        print(" attained at E_", divisors[0]+1, end = '')
        
    number_jn_diff_support = 1
    
    # Print a presentation of the multiplier ideal
    amount_of_same_jn = np.count_nonzero(JumpingNumberList[:,number_functions_monomial + number_divisors] == \
                                         JumpingNumberList[0,number_functions_monomial + number_divisors])

    generators = min_poset(JumpingNumberList[amount_of_same_jn : , : number_functions_monomial])
    print('\n')
    print('J(' + str(jump_fraction) + "/" + str(Matrix_orders_vanishing[supp][number_functions]) + \
          ' · C) = ( ', end = '')
    # Print the generators
    for gen in range(len(generators)):
        print_monomial(number_functions_monomial,generators[gen])
        if gen < len(generators) - 1:
            print(', ', end = '')
    print(')')

            
    
    # Print rest of jumping numbers
    for i in range(1,number_dif_jn):
        # Support of current jumping number
        divisors = np.where(JumpingNumberList[i][number_functions_monomial : number_functions_monomial + number_divisors] == \
                            JumpingNumberList[i][number_functions_monomial + number_divisors])

        # If the jumping number is different from previous one
        if JumpingNumberList[i][number_functions_monomial + number_divisors] != \
        JumpingNumberList[i-1][number_functions_monomial + number_divisors]:
            print("\n")
            # Print the monomial
            print_monomial(number_functions_monomial,JumpingNumberList[i])
            # or, if one wants to print 0 coordinates:
            #for exp in range(number_functions_monomial):
                #print("z_" + str(exp) + "^" + str(int(JumpingNumberList[i][exp])) + " ", end = '')

            # Print the jumping number
            print(" generates the jumping number " + \
                  str(JumpingNumberList[i][number_functions_monomial + number_divisors]), end = '')

            # Print the fractions corresponding to the jumping number and divisors in the support
            for supp in divisors[0]:
                jump_fraction = int(round(JumpingNumberList[i][number_functions_monomial + number_divisors] * \
                                          Matrix_orders_vanishing[supp][number_functions]))
                print(" = " + str(jump_fraction) + "/" + str(Matrix_orders_vanishing[supp][number_functions]), end = '')

            # Print the support of the jumping number (the set of divisors)
            print(" at divisors E_", divisors[0]+1)

            number_jn_diff_support += 1

        # If jumping number is equal to previous one but support is different    
        elif JumpingNumberList[i][number_functions_monomial + number_divisors] == \
        JumpingNumberList[i-1][number_functions_monomial + number_divisors]:
            
            print("  ", end = '')
            # Print the monomial
            print_monomial(number_functions_monomial,JumpingNumberList[i])

            # Print the jumping number
            print(" also generates " + \
                  str(JumpingNumberList[i][number_functions_monomial + number_divisors]), end = '')

            # Print the fractions corresponding to the jumping number and divisors in the support
            for supp in divisors[0]:
                jump_fraction = int(round(JumpingNumberList[i][number_functions_monomial + number_divisors] * \
                                          Matrix_orders_vanishing[supp][number_functions]))
                print(" = " + str(jump_fraction) + "/" + str(Matrix_orders_vanishing[supp][number_functions]), end = '')

            # Print the support of the jumping number (the set of divisors)
            print(" at divisors E_", divisors[0]+1, end = '')

            print(" (Dif sup).")
            number_jn_diff_support += 1
            
        # If jumping number different from next one, print presentation of the multiplier ideal    
        if i < number_dif_jn and \
        MonomioGen[i][number_functions_monomial + number_divisors] != \
        MonomioGen[i+1][number_functions_monomial + number_divisors]:
            
            generators = min_poset(MonomioGen[i+1 : , : number_functions_monomial])
            print('\n')
            print('J(' + str(jump_fraction) + "/" + str(Matrix_orders_vanishing[supp][number_functions]) + \
                  ' · C) = ( ', end = '')
            # Print the generators
            for gen in range(len(generators)):
                print_monomial(number_functions_monomial,generators[gen])
                if gen < len(generators) - 1:
                    print(', ', end = '')
            print(')')
            


    print("\nThe number of monomial candidates < 1 was: " + str(number_dif_jn))
    print("The amount of different jumping numbers < 1 is: " + str(number_jn))
    print("The amount of jumping numbers (with different support) < 1 is: " + str(number_jn_diff_support))
    if bool_curve_red != 'n' and bool_curve_red != 'N':
        print("Amount of jumping numbers < 1 counted with multiplicity: " + str(number_jn_multiplicity))

    
(bool_curve_red,number_exceptional_divisors,number_divisors,number_functions,number_functions_monomial,number_components,components_weight,Matrix_orders_vanishing) = \
get_curve_data()

if bool_curve_red != 'n' and bool_curve_red != 'N':
    number_jn_multiplicity = \
    get_Milnor(number_divisors,number_functions,number_functions_monomial,number_components,Matrix_orders_vanishing)

(number_dif_jn,number_jn,JumpingNumberList,MonomioGen) = \
get_jumping_numbers(number_exceptional_divisors,number_divisors,number_functions,number_functions_monomial,number_components,components_weight,Matrix_orders_vanishing)

print_jumping_numbers(number_divisors,number_functions,number_functions_monomial,number_components,\
                      Matrix_orders_vanishing,number_jn_multiplicity,number_dif_jn,number_jn,JumpingNumberList,MonomioGen)
    
    

# Example ThreeDiv
# 3, 5, 3, 2
# [[ 2 3 6 12 9 21 5] [ 4 6 15 30 18 48 13] [ 3 5 9 18 15 33 8]] 
# 2,3

# Example
# 4, 10, 4, 8
# [2 1 1 1 2 2 2 2 1 1 3] 
# [1 2 2 2 1 1 1 1 2 2 3] 
# [1 2 3 2 1 1 1 1 3 2 4] 
# [1 2 2 3 1 1 1 1 2 3 4]
# 3, 4, 1, 1, 1, 1, 3, 4

How many relevant exceptional divisors does the curve have?
4
How many functions do you need to generate the maximal contact completion of the curve?
12
Consider the dual graph of the minimal embedded resolution of the curve. How many functions do you need to complete it (one for each exceptional divisor of valency one)?
4
How many components does the curve have?
8
Is the curve C reduced? y/n
y

Now, for each divisor, introduce the orders of vanishing and its log-discrepancy.
Let z_0 represent the transversal branch to the curve and the components of the curve be the last 8: z_4, z_11. Also, let z_0, ..., z_4 be the curves needed to complete the dual graph. For each divisor: Introduce the orders of vanishing of the z_i separated by single spaces and press enter, then introduce the log-discrepancy and press enter again. 
The curve is represented by z_12.
The divisors vary from E_1 to E_4

Enter the orders of vanishing along E_1
2  1  1  1  2  2  2  2  1  1  1  1
Introduce the log-discre



J(9/14 · C) = ( z_0^1 z_2^3 , z_1^1 z_2^3 z_3^1 , z_0^1 z_2^2 z_3^1 , z_3^5 , z_2^4 z_3^1 , z_1^1 z_3^4 , z_1^3 z_2^2 , z_1^3 z_2^1 z_3^1 , z_1^1 z_2^1 z_3^3 , z_1^4 z_3^1 , z_0^1 z_3^3 , z_2^3 z_3^2 , z_1^1 z_2^2 z_3^2 , z_1^1 z_2^4 , z_1^4 z_2^1 , z_2^2 z_3^3 , z_0^1 z_2^1 z_3^2 , z_1^3 z_3^2 , z_1^5 , z_1^2 z_3^3 , z_2^1 z_3^4 , z_0^1 z_1^1 z_2^1 z_3^1 , z_1^2 z_2^1 z_3^2 , z_0^1 z_1^1 z_2^2 , z_0^1 z_1^3 , z_1^2 z_2^3 , z_1^2 z_2^2 z_3^1 , z_2^5 , z_0^1 z_1^2 z_3^1 , z_0^1 z_1^2 z_2^1 , z_0^1 z_1^1 z_3^2 , z_0^2 z_2^2 , z_0^6 , z_0^4 z_3^1 , z_0^4 z_1^1 , z_0^4 z_2^1 , z_0^2 z_1^1 z_2^1 , z_0^2 z_1^1 z_3^1 , z_0^2 z_1^2 , z_0^2 z_3^2 , z_0^2 z_2^1 z_3^1 )


z_0^1 z_2^3  generates the jumping number 0.6666666666666666 = 8/12 at divisors E_ [1]
  z_1^1 z_2^3 z_3^1  also generates 0.6666666666666666 = 8/12 at divisors E_ [1] (Dif sup).
  z_0^1 z_2^2 z_3^1  also generates 0.6666666666666666 = 8/12 at divisors E_ [1] (Dif sup).
  z_3^5  also generates 0.6666666666666666 = 8/12 at divi



J(9/12 · C) = ( z_0^3 z_3^2 , z_0^3 z_1^2 , z_0^7 , z_0^3 z_2^2 , z_0^5 z_1^1 , z_0^5 z_2^1 , z_0^5 z_3^1 , z_0^3 z_1^1 z_2^1 , z_0^3 z_1^1 z_3^1 , z_1^3 z_2^4 , z_2^2 z_3^5 , z_1^4 z_2^1 z_3^2 , z_0^1 z_2^3 z_3^2 , z_2^5 z_3^2 , z_0^2 z_3^3 , z_0^2 z_2^1 z_3^2 , z_2^1 z_3^6 , z_0^3 z_2^1 z_3^1 , z_0^1 z_2^2 z_3^3 , z_1^4 z_2^2 z_3^1 , z_1^2 z_2^5 , z_0^2 z_2^2 z_3^1 , z_0^1 z_2^5 , z_1^4 z_2^3 , z_0^2 z_2^3 , z_1^2 z_2^4 z_3^1 , z_0^1 z_1^2 z_2^3 , z_1^6 z_2^1 , z_0^1 z_2^4 z_3^1 , z_0^1 z_1^2 z_2^2 z_3^1 , z_1^4 z_3^3 , z_2^3 z_3^4 , z_0^1 z_1^3 z_2^2 , z_1^3 z_2^2 z_3^2 , z_0^2 z_1^2 z_3^1 , z_0^1 z_1^3 z_2^1 z_3^1 , z_2^6 z_3^1 , z_0^1 z_1^4 z_3^1 , z_0^1 z_1^4 z_2^1 , z_1^3 z_2^1 z_3^3 , z_1^1 z_2^4 z_3^2 , z_1^1 z_2^5 z_3^1 , z_1^3 z_3^4 , z_0^1 z_1^5 , z_1^1 z_2^2 z_3^4 , z_0^1 z_3^5 , z_1^1 z_2^1 z_3^5 , z_1^7 , z_1^1 z_3^6 , z_0^1 z_2^1 z_3^4 , z_0^1 z_1^3 z_3^2 , z_1^2 z_2^3 z_3^2 , z_1^3 z_2^3 z_3^1 , z_1^5 z_2^1 z_3^1 , z_1^6 z_3^1 , z_0^1 z_1^1 z_3^4 , z_0^2 z_1^2 z_2^1 



J(10/12 · C) = ( z_0^6 z_3^1 , z_0^4 z_1^1 z_2^1 , z_0^4 z_2^2 , z_0^6 z_1^1 , z_0^8 , z_0^4 z_1^1 z_3^1 , z_0^4 z_1^2 , z_0^4 z_3^2 , z_0^6 z_2^1 , z_0^1 z_1^5 z_3^1 , z_0^1 z_1^5 z_2^1 , z_1^6 z_3^2 , z_1^3 z_2^5 , z_0^1 z_1^4 z_2^2 , z_1^8 , z_0^1 z_1^6 , z_0^1 z_1^4 z_2^1 z_3^1 , z_1^5 z_2^1 z_3^2 , z_0^1 z_1^4 z_3^2 , z_1^1 z_2^3 z_3^4 , z_1^3 z_2^4 z_3^1 , z_1^5 z_3^3 , z_0^2 z_1^2 z_2^1 z_3^1 , z_1^7 z_2^1 , z_0^4 z_2^1 z_3^1 , z_1^7 z_3^1 , z_0^2 z_2^4 , z_0^2 z_2^3 z_3^1 , z_1^4 z_2^4 , z_0^2 z_1^3 z_2^1 , z_1^6 z_2^1 z_3^1 , z_1^6 z_2^2 , z_1^4 z_2^3 z_3^1 , z_0^2 z_2^2 z_3^2 , z_0^2 z_1^3 z_3^1 , z_1^3 z_2^3 z_3^2 , z_0^2 z_2^1 z_3^3 , z_1^4 z_2^2 z_3^2 , z_0^2 z_1^1 z_3^3 , z_2^3 z_3^5 , z_0^2 z_3^4 , z_1^4 z_2^1 z_3^3 , z_2^5 z_3^3 , z_0^2 z_1^1 z_2^1 z_3^2 , z_1^4 z_3^4 , z_0^2 z_1^1 z_2^2 z_3^1 , z_1^1 z_2^4 z_3^3 , z_0^2 z_1^1 z_2^3 , z_1^5 z_2^3 , z_0^2 z_1^2 z_2^2 , z_0^2 z_1^4 , z_1^5 z_2^2 z_3^1 , z_2^4 z_3^4 , z_0^1 z_1^1 z_2^5 , z_0^2 z_1^2 z_3^2 , z_0^1 z_3^6 ,



J(11/12 · C) = ( z_0^3 z_2^3 , z_0^7 z_1^1 , z_0^7 z_2^1 , z_0^7 z_3^1 , z_0^9 , z_0^3 z_3^3 , z_0^3 z_1^1 z_3^2 , z_0^3 z_1^1 z_2^2 , z_0^5 z_3^2 , z_0^3 z_1^2 z_2^1 , z_0^5 z_1^1 z_2^1 , z_0^5 z_1^2 , z_0^3 z_1^3 , z_0^3 z_1^2 z_3^1 , z_0^5 z_1^1 z_3^1 , z_0^5 z_2^2 , z_0^1 z_1^2 z_2^1 z_3^4 , z_0^2 z_1^1 z_2^3 z_3^1 , z_0^1 z_2^7 , z_0^1 z_1^1 z_2^5 z_3^1 , z_0^1 z_1^1 z_2^6 , z_0^1 z_1^1 z_2^3 z_3^3 , z_1^5 z_2^4 , z_1^6 z_3^3 , z_0^2 z_2^3 z_3^2 , z_0^1 z_1^1 z_2^4 z_3^2 , z_0^2 z_1^1 z_2^4 , z_0^2 z_2^4 z_3^1 , z_0^2 z_2^5 , z_0^2 z_1^1 z_2^2 z_3^2 , z_0^1 z_1^2 z_2^2 z_3^3 , z_0^1 z_1^1 z_3^6 , z_0^2 z_1^1 z_2^1 z_3^3 , z_0^1 z_1^2 z_3^5 , z_1^5 z_2^2 z_3^2 , z_1^5 z_2^1 z_3^3 , z_2^3 z_3^6 , z_0^1 z_1^1 z_2^1 z_3^5 , z_2^7 z_3^2 , z_0^2 z_1^1 z_3^4 , z_2^8 z_3^1 , z_0^1 z_1^1 z_2^2 z_3^4 , z_1^5 z_2^3 z_3^1 , z_0^1 z_1^3 z_2^3 z_3^1 , z_1^6 z_2^1 z_3^2 , z_1^7 z_2^2 , z_0^1 z_1^6 z_2^1 , z_0^1 z_1^6 z_3^1 , z_0^1 z_1^5 z_2^2 , z_0^1 z_1^5 z_2^1 z_3^1 , z_1^1 z_2^1 z_3^7 , z_0

In [6]:
MonomioGen[:6]

array([[0.        , 0.        , 0.        , 0.        , 0.25      ,
        0.25      , 0.28571429, 0.28571429, 0.25      ],
       [0.        , 0.        , 0.        , 1.        , 0.33333333,
        0.41666667, 0.42857143, 0.5       , 0.33333333],
       [1.        , 0.        , 0.        , 0.        , 0.41666667,
        0.33333333, 0.35714286, 0.35714286, 0.33333333],
       [0.        , 1.        , 0.        , 0.        , 0.33333333,
        0.41666667, 0.42857143, 0.42857143, 0.33333333],
       [0.        , 0.        , 1.        , 0.        , 0.33333333,
        0.41666667, 0.5       , 0.42857143, 0.33333333],
       [0.        , 1.        , 1.        , 0.        , 0.41666667,
        0.58333333, 0.64285714, 0.57142857, 0.41666667]])